In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
import swifter
import pandas as pd
import re
import string

stop_words = list(set(stopwords.words('english')))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[{}0-9]'.format(string.punctuation), ' ', text)
    text=re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df=pd.DataFrame({"content":newsgroups["data"]})

df["content"]=df["content"].swifter.apply(lambda x: preprocess_text(x))
df['content_length'] = df['content'].str.len()

df = df[df['content_length'] > 100]
df = df[df['content_length'] < 2000]

df=df[["content"]].reset_index(drop=True).reset_index().rename(columns={"index":"id"})
documents=df.content.to_list()

Pandas Apply:   0%|          | 0/18846 [00:00<?, ?it/s]

In [3]:
from ctc.main import Auto_CTC
x=Auto_CTC(segments_length=15, min_segment_length=5, segment_step=10,device="mps")
docs=documents[:1]
x.segmenting_documents(docs)
x.create_cpmi_tree()
#x.load_cpmi_tree()
topics=[["game","play"],["man","devil"]]
x.ctc_cpmi(topics)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2/2 [00:00<00:00, 54120.05it/s]


0.38507628440856934

In [4]:
from ctc.main import Semi_auto_CTC
openai_key="YOUR OPENAI KEY"
y=Semi_auto_CTC(openai_key,topics)
y.ctc_intrusion(),y.ctc_rating()

(0.8166666666666667, 2.5)